In [18]:
import numpy as np 
import pandas as pd 

def objective_function(x, test, prob):
    a, b, c = x
    return np.sum((a * test[0] + b * test[1] + c * test[2] - prob) ** 2)

def constraint(x):
    return np.sum(x) - 1

# create a random np vector of probabilities
prob = np.random.random(500)

test = np.random.random((3,500))

x0 = np.array([1/3, 1/3, 1/3])

In [19]:
objective_function(x0, test, prob)

53.964975079416796

In [20]:
import numpy as np
from scipy.optimize import minimize

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1

result = minimize(objective_function, x0, args=(test, prob), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("c =", result.x[2])
print("Objective function value:", result.fun)

Optimal values:
a = 0.3520261659868651
b = 0.33936192866185355
c = 0.30861190535128136
Objective function value: 53.92316611331649
